In [1]:
from scipy import sparse

def gen_rand_sparse_mat(m, n, density):
    M = sparse.rand(m, n, density=density)
    M.data[:] = 1
    return M

X = gen_rand_sparse_mat(10, 10, 0.1)
print X.toarray()

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [61]:
#TAGS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
ITEMS = range(1000)
TAGS = range(100)
ITEM_COUNT = len(ITEMS)
TAG_COUNT = len(TAGS)
DENSITY = 0.15

# X: tag matrix
# T: maximun number of questions
# GAMMA: discount factor
X = gen_rand_sparse_mat(ITEM_COUNT, TAG_COUNT, DENSITY)
T = 5
GAMMA = 0.75

print X.toarray()

[[ 0.  0.  0. ...,  0.  1.  1.]
 [ 1.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]


In [47]:
import numpy as np
from scipy import sparse

def questionSellectionAlgo_greedy(X, w, qs):
    X_qs = X.toarray()[:, qs]
    E = np.dot(w, X_qs) / np.dot(w, X_qs).sum()
    q = np.argmin(np.fabs(E - 0.5))
        
    return qs[q]
    
print questionSellectionAlgo_greedy(X, np.ones(ITEM_COUNT), range(TAG_COUNT))

8


In [29]:

def questionSellectionAlgo_ihs_exhaustion(X, GAMMA, w, qs, k):
    ls = (((np.array(range(2**k))[:,None] & (1 << np.arange(k)))) > 0).astype(int)
    qks = [qk for qk in itertools.combinations(qs, k)]
    
    min_ls = []
    for qk in qks:   
        X_qk = X.toarray()[:, [qs[q] for q in qk]].astype(int)
        
        ls_ds = []
        for l in ls:
            dcs = [np.bitwise_xor(l, X_qki).sum() for X_qki in X_qk]
            d = (w*[1 - GAMMA**dc for dc in dcs]).sum()
            ls_ds.append(d)
        min_ls.append(np.min(ls_ds))
    
    Q = np.argmax(min_ls)
    
    return [qs[q] for q in qks[Q]]

# may take about 2 min for k=2 on 4G RAM
#print questionSellectionAlgo_ihs_exhaustion(X, GAMMA, np.ones(ITEM_COUNT), range(TAG_COUNT), 2)

[47, 93]


In [500]:
import random

def jump_prob(e, enew, T):
    if enew < e: 
        return 1
    else:
        return np.exp((e-enew)/T)

def compute_min_d(X_qk, GAMMA, w, ls):
    ls_ds = []
    for l in ls:
        dcs = [np.bitwise_xor(l, X_qk.astype(int)).sum()]
        d = (w*[1 - GAMMA**dc for dc in dcs]).sum()
        ls_ds.append(d)
    return np.min(ls_ds)

def questionSellectionAlgo_ihs_sa(X, GAMMA, w, qs, k, I, T):
    ls = (((np.array(range(2**k))[:,None] & (1 << np.arange(k)))) > 0).astype(int)
    qk = np.random.choice(qs, k, replace=False)
    X_qk = X.toarray()[:, qk]
    min_d = compute_min_d(X_qk, GAMMA, w, ls)
    t = T
    
    qk_best = qk
    for i in range(I):
        qk_n = np.array(qk)
        qk_n[np.random.choice(k)] = np.random.choice(np.setdiff1d(qs, qk))
        X_qk_n = X.toarray()[:, qk_n]
        min_d_n = compute_min_d(X_qk_n, GAMMA, w, ls)
        
        t = t*0.9
        if jump_prob(min_d, min_d_n, t) > np.random.rand(1):
            qk = qk_n
        if min_d_n > min_d:
            min_d = min_d_n
            qk_best = qk_n
     
    return qk_best

print questionSellectionAlgo_ihs_sa(X, GAMMA, np.ones(ITEM_COUNT), range(TAG_COUNT), 2, 1000, 100)

[16 21]


In [ ]:

def interactiveRecommandation(X, T, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    t = 0
    for t in range(T):
        q = questionSellectionAlgo_greedy(X, w, r_qs)
        r_qs = r_qs[r_qs != q]
        
        X_q = X.toarray()[:, q]
        
        ans = raw_input('has ' + str(TAGS[q]) + '? (0/1): ')
        g = np.ones(ITEM_COUNT)
        if ans == '1':
            g = X_q
        else:
            g = (1 - X_q)
        g[g < 1] = GAMMA
        w = w*g
        
        print X_q
        print g
        print 'W: ', w
    return w

interactiveRecommandation(X, T, GAMMA)

In [ ]:
""" test case
from scipy.sparse import coo_matrix

X = coo_matrix(np.matrix([
    [1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
    [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
    [0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.],
    [0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
    [0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.],
    [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
    [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
    [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
    [0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0.]
]))
TAG_COUNT = ITEM_COUNT = 10
GAMMA = 0.5
T = 10
"""